In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'test.csv', 'sample_submission.csv']


In [3]:
# Importing necessary Packages
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import  cross_val_score
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import sklearn
import numpy as np
from nltk import word_tokenize
from collections import Counter

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#Reading the Training Dataset
dftr = pd.read_csv("../input/train.csv")
#Reading the Testing Dataset
dfts = pd.read_csv("../input/test.csv")

## Text Preprocessing

In [5]:
STOPWORDS =  stopwords.words('english')

In [6]:
#Defining a txtprepros function
def txtprepros(x):
    x = " ".join([word.lower() for word in x.split() ])                              #Lowercasing all letters
    x = re.sub('[^A-Za-z0-9]+'," ",x)                                                # Deleting some symbols by space
    x  = " ".join([word for word in x.split() if word not in STOPWORDS])             #Removing Stopwords
    return x       

In [7]:
dftr["comment_text"] = dftr["comment_text"].apply(lambda x: txtprepros(x))
dfts["comment_text"] = dfts["comment_text"].apply(lambda x: txtprepros(x))

In [8]:
dftr.comment_text.isnull().sum()

0

In [9]:
dfts.comment_text.isnull().sum()

0

In [10]:
train_words=dftr.comment_text.map(lambda x:len(x.split()))
test_words=dfts.comment_text.map(lambda x:len(x.split()))

In [11]:
# Training words description
train_words.describe()

count    159571.000000
mean         35.852718
std          55.698275
min           0.000000
25%           9.000000
50%          19.000000
75%          39.000000
max        1250.000000
Name: comment_text, dtype: float64

In [12]:
# Testing words description
test_words.describe()

count    153164.000000
mean         32.883419
std          58.756059
min           0.000000
25%           7.000000
50%          16.000000
75%          34.000000
max        1667.000000
Name: comment_text, dtype: float64

In [13]:
#Y-Variables for Train Data set
YT = np.array(dftr.iloc[:,2:])

## Building a LSTM Model

In [14]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.layers import Embedding
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [15]:
# Deteremining Train Variables
tk = Tokenizer()
tk.fit_on_texts(dftr.comment_text)
vocab_size = len(tk.word_index) + 1
# integer encode the documents
encoded_docs = tk.texts_to_sequences(dftr.comment_text)
# pad documents to a max length of 99th percentile
max_length = int((train_words.quantile([.99])).values[0])
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post',truncating='post')
print('shape of Train input : ',padded_docs.shape)

shape of Train input :  (159571, 312)


In [16]:
#Determing Test Variables
encoded_docs1 = tk.texts_to_sequences(dfts.comment_text)
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length, padding='post',truncating='post')
print('shape of Test input sequence :', padded_docs1.shape)

shape of Test input sequence : (153164, 312)


In [17]:
# Creating the Model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
main_input = Input(shape=(max_length,), dtype='int32', name='main_input')
# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=50, input_dim=vocab_size, input_length=max_length)(main_input)
x = LSTM(32)(x)   #x = Flatten()(x)
toxic = Dense(1, activation='sigmoid', name='toxic_output')(x)
severe_toxic = Dense(1, activation='sigmoid', name='severe_toxic_output')(x)
obscene = Dense(1, activation='sigmoid', name='obscene_output')(x)
threat = Dense(1, activation='sigmoid', name='threat_output')(x)
insult = Dense(1, activation='sigmoid', name='insult_output')(x)
identity_hate = Dense(1, activation='sigmoid', name='identity_hate_output')(x)
model = Model(inputs=[main_input], outputs=[toxic, severe_toxic,obscene,threat,insult,identity_hate ])
model.compile(optimizer='rmsprop',
              loss={'toxic_output': 'binary_crossentropy', 'severe_toxic_output': 'binary_crossentropy',
                    'obscene_output': 'binary_crossentropy', 'threat_output': 'binary_crossentropy',
                   'insult_output': 'binary_crossentropy', 'identity_hate_output': 'binary_crossentropy'},
              loss_weights={'toxic_output': 1, 'severe_toxic_output': 1,
                    'obscene_output':1, 'threat_output': 1,
                   'insult_output': 1, 'identity_hate_output': 1},metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 312)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 312, 50)      9105750     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           10624       embedding_1[0][0]                
__________________________________________________________________________________________________
toxic_output (Dense)            (None, 1)            33          lstm_1[0][0]                     
__________________________________________________________________________________________________
severe_tox

In [18]:
model.fit({'main_input': padded_docs},
          {'toxic_output': YT[:,0], 'severe_toxic_output':  YT[:,1],
                    'obscene_output': YT[:,2], 'threat_output':  YT[:,3],
                   'insult_output':  YT[:,4], 'identity_hate_output':  YT[:,5]},
          epochs=1, batch_size=32)

Epoch 1/1
159571/159571 [==============================] - 3526s 22ms/step - loss: 0.8547 - toxic_output_loss: 0.3167 - severe_toxic_output_loss: 0.0571 - obscene_output_loss: 0.2080 - threat_output_loss: 0.0224 - insult_output_loss: 0.1975 - identity_hate_output_loss: 0.0528 - toxic_output_acc: 0.9042 - severe_toxic_output_acc: 0.9901 - obscene_output_acc: 0.9470 - threat_output_acc: 0.9969 - insult_output_acc: 0.9507 - identity_hate_output_acc: 0.9909


## Prediction on test Datasets

In [19]:
y_pred = model.predict(padded_docs1)

## Submission

In [20]:
df_sub=pd.read_csv('../input/sample_submission.csv',
                     sep=',', engine='c')

In [21]:
df_sub.toxic=y_pred[0]
df_sub.severe_toxic=y_pred[1]
df_sub.obscene=y_pred[2]
df_sub.threat=y_pred[3]
df_sub.insult=y_pred[4]
df_sub.identity_hate=y_pred[5]

In [30]:
dfsub = df_sub.copy()

In [26]:
list(dfsub)

['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [32]:
for i in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    dfsub[i] = dfsub[i] / (dfsub[i].max() - dfsub[i].min())

In [34]:
for i in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    dfsub[i] = np.where(dfsub[i]>dfsub[i].mean(),1,0)

In [40]:
df_sub.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000
mean,0.093171,0.010875,0.046685,0.002644,0.046631,0.006532
std,0.017763,0.019654,0.016877,0.014358,0.018661,0.016511
min,0.078705,0.005107,0.034659,0.001007,0.037211,0.002726
25%,0.092341,0.010123,0.046125,0.002115,0.045959,0.005893
50%,0.092341,0.010123,0.046125,0.002115,0.045959,0.005893
75%,0.092341,0.010123,0.046125,0.002115,0.045959,0.005893
max,0.723256,0.724981,0.645952,0.593561,0.707177,0.611279


In [42]:
df_sub.to_csv('LSTM.csv',index=False)

In [37]:
df_sub.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000,153164.000000
mean,0.004126,0.004192,0.004642,0.002357,0.004629,0.003500
std,0.064104,0.064607,0.067975,0.048491,0.067880,0.059053
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
